# Extract - 001
- Extract all the stocks
- Extract the stock sectors
- Extract the fundamentals indicators

# 1 - Libs and Packages

In [2]:
# packages
# data manipulation
import pandas as pd
import datetime as dt

# using the scrapping lib
import re
import requests
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse

# lib to passe cookies
import http.cookiejar

# database
import sqlite3

# 2 - Extract

In [3]:
# url if the page to scrape
# Create a stock to insert inside the url
stock = 'TRPL4'

url = f'https://www.fundamentus.com.br/detalhes.php?papel={stock}'

In [ ]:
# create a conn to insert inside the database
conn = sqlite3.connect(r'../database/database.sqlite3')

In [4]:
# create a token to pass the cookies
cookie_jar = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookie_jar))
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'),
                         ('Accept', 'text/html, text/plain, text/css, text/sgml, */*;q=0.01')]

In [5]:
# Send http request
html = opener.open(url)
html_content = html.read().decode('ISO-8859-1')

In [6]:
# find the pattern of the table inside the html content
pattern = re.compile('<table class="w728">.*</table>',re.DOTALL)
# apply the pattern inside the html
final_soup = re.findall(pattern,html_content)[0]

In [7]:
soup = BeautifulSoup(final_soup,'lxml')

In [8]:
# create a empty list to store the data
data = []
for dt in soup.find_all("span",attrs={'class':'txt'}):
    data.append(dt.text)

In [9]:
data

['Papel',
 'TRPL4',
 'Cotação',
 '25,13',
 'Tipo',
 'PN N1',
 'Data últ cot',
 '31/05/2024',
 'Empresa',
 'TRANSMISSÃO PAULISTA PN N1',
 'Min 52 sem',
 '19,68',
 'Setor',
 'Energia Elétrica',
 'Max 52 sem',
 '27,75',
 'Subsetor',
 'Energia Elétrica',
 'Vol $ méd (2m)',
 '90.173.100',
 'Valor de mercado',
 '16.557.700.000',
 'Últ balanço processado',
 '31/03/2024',
 'Valor da firma',
 '24.184.000.000',
 'Nro. Ações',
 '658.883.000',
 'Oscilações',
 'Indicadores fundamentalistas',
 'Dia',
 'P/L',
 '6,05',
 'LPA',
 '4,15',
 'Mês',
 'P/VP',
 '0,92',
 'VPA',
 '27,34',
 '30 dias',
 'P/EBIT',
 '\n4,84',
 'Marg. Bruta',
 '\n55,7%',
 '12 meses',
 'PSR',
 '\n2,50',
 'Marg. EBIT',
 '\n51,7%',
 '2024',
 'P/Ativos',
 '\n0,44',
 'Marg. Líquida',
 '\n42,2%',
 '2023',
 'P/Cap. Giro',
 '\n3,62',
 'EBIT / Ativo',
 '9,0%',
 '2022',
 'P/Ativ Circ Liq',
 '\n-1,31',
 'ROIC',
 '\n9,8%',
 '2021',
 'Div. Yield',
 '8,8%',
 'ROE',
 '\n15,2%',
 '2020',
 'EV / EBITDA',
 '\n7,01',
 'Liquidez Corr',
 '\n3,08',
 '201

In [10]:
# create a empty list to store the columns
data = []
for dt in soup.find_all("span",attrs={'class':'txt'}):
    data.append(dt.text)

In [50]:
# create a empty list to store the columns names
column_names = []
column_names.append(data[0:len(data):2])

In [51]:
# get until the nine line and get the real name columns
column_names_clean = []
for column in column_names:
    column_names_clean.append(column[:9])

In [52]:
# create a empty list to return the rows from nine lines
rows = []
rows.append(data[1:len(data):2])
rows_clean = []
for row in rows:
    rows_clean.append(str(row[:9]))

In [54]:
# Using eval function to evaluate the python expression as 'string' and return the value as an integer
rows_clean = eval(rows_clean[0])

In [77]:
final_data = pd.DataFrame(
    [rows_clean], columns=column_names_clean
)

# 3 - Transform

In [78]:
# take the Multindex of the columns level
final_data.columns = final_data.columns.map(''.join)

In [79]:
# clean all the columns names
final_data.columns = final_data.columns.str.replace(' ','_').str.replace('ú','u').str.replace('ã','a').str.replace('ç','c').str.lower()

In [80]:
# clean the data for datetime
final_data['data_ult_cot'] = final_data['data_ult_cot'].astype('datetime64[ns]')

In [84]:
# clean the decimal point for convert to float
final_data['cotacao'] = final_data['cotacao'].str.replace(',','.')

In [85]:
# convert to float
final_data['cotacao'] = final_data['cotacao'].astype('float64')

In [87]:
final_data['max_52_sem'] = final_data['max_52_sem'].str.replace(',','.')
final_data['max_52_sem'] = final_data['max_52_sem'].astype('float64')
final_data['min_52_sem'] = final_data['min_52_sem'].str.replace(',','.')
final_data['min_52_sem'] = final_data['min_52_sem'].astype('float64')

In [91]:
final_data['data_extract'] = dt.date.today()

In [92]:
final_data.to_sql('stock_prices_daily',
                  con=conn,
                  schema=None,
                  index=False)

1